In [262]:
import random
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,AveragePooling2D
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [263]:
class LayerFactory:
    def __init__(self):
        self.func_mapping = {
            0: 'get_random_conv_2d',
            1: 'get_random_max_pooling',
            2: 'get_random_avg_pooling',
            3: 'get_random_dropout'
        }
        self.num_of_layer_types = len(self.func_mapping)
        
    def get_random(self):
        layer_type = self.func_mapping[random.randrange(0, self.num_of_layer_types)]
        layer_func = getattr(self, layer_type)
        return layer_func()
        
    def get_random_conv_2d(self):
        return Conv2D(self.randomFiltersConv2D(), kernel_size = self.randomKernelSizeConv2D(), strides = self.randomStrideConv2D(), padding = self.randomPaddingConv2D(), activation = self.randomActivationConv2D())
    def get_random_max_pooling(self):
        return MaxPooling2D(pool_size = self.randomPoolSize())
    def get_random_avg_pooling(self):
        return AveragePooling2D(pool_size = self.randomPoolSize())
    def get_random_dropout(self):
        return Dropout(random.uniform(0,1))
    

    def randomFiltersConv2D(self):
        return random.choice([32, 64])
    
    def randomKernelSizeConv2D(self):
        return random.choice([(3, 3),(2,2)])
    
    def randomStrideConv2D(self):
        return random.choice([(1, 1),(2,2)])
    
    def randomPaddingConv2D(self):
        return random.choice(["same"])
    
    def randomActivationConv2D(self):
        return random.choice(["relu"])
    
    def randomPoolSize(self):
        return random.choice([(2,2)])

In [264]:
class Individual:
    def __init__(self, input_shape):
        self.code = []
        self.input_shape = input_shape
        self.num_middle_layers = 0
        self.fitness = 0

    
    def from_configured_layers(self, layers):
        self.code = [self._resolve_layer(layer) for layer in layers]
        
        self.num_middle_layers = len(layers)
        self.fitness = 0 #reset fitness just in case
                    
    def fill_random(self, num_middle_layers, lf):
        self.num_middle_layers = num_middle_layers 
        for i in range(self.num_middle_layers):
            self.code.append(lf.get_random())
            
    def calculate_fitness(self, X_train, y_train, X_validation, y_validation):
        
        model = self._to_model()
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, batch_size=128, epochs=9, validation_split=0.2)
        model.evaluate(X_validation, y_validation)
        
        y_pred = np.argmax(model.predict(X_validation),axis = 1)
        y_true = np.argmax(y_validation, axis = 1)
        
        self.fitness = accuracy_score(y_true, y_pred)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def __str__(self):
        modelString = ""
        for i in range(len(self.dictionary)):
            modelString+=" "
            modelString+=str(i)
        
        return str(self.dictionary)
            
    def _to_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=self.input_shape))
        
        for layer in self.code:
            model.add(layer)
            
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(units = 10 ,activation = 'softmax'))
        
        return model
            
    def _resolve_layer(self, layer):
        layer_config = layer.get_config()
        if type(layer) == Conv2D:
            return Conv2D(filters = layer_config['filters'], kernel_size = layer_config['kernel_size'], strides = layer_config['strides'], padding = layer_config['padding'], activation = layer_config['activation'])
        elif type(layer) == MaxPooling2D:
            return MaxPooling2D(pool_size = layer_config['pool_size'])
        elif type(layer) == AveragePooling2D:
            return AveragePooling2D(pool_size = layer_config['pool_size'])
        elif type(layer) == Dropout:
            return Dropout(layer_config['rate'])
        else:
            print("Unsupported layer type", type(layer))
            return None

In [265]:
def selection(population):
    TOURNAMENT_SIZE = 5
    participants = random.sample(population, TOURNAMENT_SIZE)
    winner = max(participants)
    return winner

In [266]:
def mutation(individual, lf):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = lf.get_random()

In [267]:
import copy
def crossover(parent1, parent2, child1, child2):
    num_of_layers_child1 = random.randrange(min(parent1.num_middle_layers, parent2.num_middle_layers), max(parent1.num_middle_layers, parent2.num_middle_layers) + 1)
    num_of_layers_child2 = random.randrange(min(parent1.num_middle_layers, parent2.num_middle_layers), max(parent1.num_middle_layers, parent2.num_middle_layers) + 1)
    
    children_middle_layers = parent1.code + parent2.code
    
    child1_layers = random.sample(children_middle_layers, num_of_layers_child1)
    child2_layers = random.sample(children_middle_layers, num_of_layers_child2)
    
    child1.from_configured_layers(child1_layers)
    child2.from_configured_layers(child2_layers)
    


In [268]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_size = X_train.shape[1]
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_size, img_size)
    X_test = X_test.reshape(X_test.shape[0], 1, img_size, img_size)
else:
    X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
    X_test = X_test.reshape(X_test.shape[0], img_size, img_size, 1)
X_train = X_train / 255

X_test = X_test / 255

X_train = X_train[:1000]
X_validation = X_test[550:1000]
X_test = X_test[:550]
num_classes = 10
y_train = to_categorical(y_train[:1000],num_classes)
y_validation = to_categorical(y_test[550:1000], num_classes)
y_test = to_categorical(y_test[:550], num_classes)

In [271]:
POP_SIZE = 14
NUM_GENERATIONS = 4
ELITISM_SIZE = POP_SIZE // 3
MIN_MID_LAYERS = 1
MAX_MID_LAYERS = 5
if ELITISM_SIZE % 2 == 1:
    ELITISM_SIZE -= 1

lf = LayerFactory()

population = [Individual((28, 28, 1)) for _ in range(POP_SIZE)]
newPopulation = [Individual((28, 28, 1)) for _ in range(POP_SIZE)]

for individual in population: 
    individual.fill_random(random.randrange(MIN_MID_LAYERS, MAX_MID_LAYERS), lf)
    individual.calculate_fitness(X_train, y_train, X_validation, y_validation)

Epoch 1/9
7/7 [==============================] - 0s 48ms/step - loss: 2.0626 - accuracy: 0.4000 - val_loss: 1.6280 - val_accuracy: 0.7600
Epoch 2/9
7/7 [==============================] - 0s 31ms/step - loss: 1.3378 - accuracy: 0.7987 - val_loss: 1.0409 - val_accuracy: 0.7850
Epoch 3/9
7/7 [==============================] - 0s 32ms/step - loss: 0.8045 - accuracy: 0.8475 - val_loss: 0.7420 - val_accuracy: 0.7750
Epoch 4/9
7/7 [==============================] - 0s 34ms/step - loss: 0.5406 - accuracy: 0.8687 - val_loss: 0.6034 - val_accuracy: 0.8150
Epoch 5/9
7/7 [==============================] - 0s 33ms/step - loss: 0.4101 - accuracy: 0.8825 - val_loss: 0.5419 - val_accuracy: 0.8200
Epoch 6/9
7/7 [==============================] - 0s 35ms/step - loss: 0.3442 - accuracy: 0.9038 - val_loss: 0.5364 - val_accuracy: 0.8400
Epoch 7/9
7/7 [==============================] - 0s 35ms/step - loss: 0.3034 - accuracy: 0.9125 - val_loss: 0.5220 - val_accuracy: 0.8550
Epoch 8/9
7/7 [===================

7/7 [==============================] - 0s 20ms/step - loss: 1.8741 - accuracy: 0.7437 - val_loss: 1.6804 - val_accuracy: 0.7500
Epoch 3/9
7/7 [==============================] - 0s 20ms/step - loss: 1.5029 - accuracy: 0.7962 - val_loss: 1.3110 - val_accuracy: 0.8000
Epoch 4/9
7/7 [==============================] - 0s 22ms/step - loss: 1.1279 - accuracy: 0.8188 - val_loss: 0.9972 - val_accuracy: 0.7850
Epoch 5/9
7/7 [==============================] - 0s 20ms/step - loss: 0.8318 - accuracy: 0.8350 - val_loss: 0.7957 - val_accuracy: 0.7850
Epoch 6/9
7/7 [==============================] - 0s 21ms/step - loss: 0.6361 - accuracy: 0.8612 - val_loss: 0.6663 - val_accuracy: 0.7900
Epoch 7/9
7/7 [==============================] - 0s 20ms/step - loss: 0.5231 - accuracy: 0.8650 - val_loss: 0.5979 - val_accuracy: 0.8150
Epoch 8/9
7/7 [==============================] - 0s 20ms/step - loss: 0.4452 - accuracy: 0.8850 - val_loss: 0.5734 - val_accuracy: 0.8150
Epoch 9/9
15/15 [===========================

Epoch 4/9
7/7 [==============================] - 0s 23ms/step - loss: 2.2057 - accuracy: 0.3150 - val_loss: 2.1995 - val_accuracy: 0.2750
Epoch 5/9
7/7 [==============================] - 0s 23ms/step - loss: 2.1585 - accuracy: 0.3425 - val_loss: 2.1426 - val_accuracy: 0.3400
Epoch 6/9
7/7 [==============================] - 0s 22ms/step - loss: 2.0973 - accuracy: 0.3825 - val_loss: 2.0720 - val_accuracy: 0.3650
Epoch 7/9
7/7 [==============================] - 0s 23ms/step - loss: 2.0171 - accuracy: 0.4162 - val_loss: 1.9859 - val_accuracy: 0.3900
Epoch 8/9
7/7 [==============================] - 0s 24ms/step - loss: 1.9201 - accuracy: 0.4600 - val_loss: 1.8829 - val_accuracy: 0.4300
Epoch 9/9
15/15 [==============================] - 0s 3ms/step - loss: 1.8343 - accuracy: 0.4356
Epoch 1/9
7/7 [==============================] - 0s 36ms/step - loss: 2.2269 - accuracy: 0.2325 - val_loss: 2.1511 - val_accuracy: 0.4550
Epoch 2/9
7/7 [==============================] - 0s 22ms/step - loss: 1.982

In [ ]:
experiment = 10
best_models = []

for k in range(experiment):
    for i in range(NUM_GENERATIONS):
        population.sort(reverse=True)
        if population[0].fitness >= 0.96:
            break

        newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
        for j in range(ELITISM_SIZE, POP_SIZE, 2):
            parent1 = selection(population)
            parent2 = selection(population)

            crossover(parent1, parent2, newPopulation[j], newPopulation[j+1])

            mutation(newPopulation[j], lf)
            mutation(newPopulation[j+1], lf)

            newPopulation[j].calculate_fitness(X_train, y_train, X_validation, y_validation)
            newPopulation[j + 1].calculate_fitness(X_train, y_train, X_validation, y_validation)    

        population = newPopulation
    best = max(population)
    best_model = best._to_model()
    y_pred = np.argmax(best_model.predict(X_test),axis = 1)
    y_true = np.argmax(y_test, axis = 1)
    fitness = accuracy_score(y_true, y_pred)
    best_models.append((best_model,fitness))

Epoch 1/9
7/7 [==============================] - 0s 62ms/step - loss: 2.1052 - accuracy: 0.4238 - val_loss: 1.7927 - val_accuracy: 0.7350
Epoch 2/9
7/7 [==============================] - 0s 45ms/step - loss: 1.3029 - accuracy: 0.7825 - val_loss: 1.0534 - val_accuracy: 0.7850
Epoch 3/9
7/7 [==============================] - 0s 44ms/step - loss: 0.6835 - accuracy: 0.8075 - val_loss: 0.7251 - val_accuracy: 0.7500
Epoch 4/9
7/7 [==============================] - 0s 44ms/step - loss: 0.4773 - accuracy: 0.8450 - val_loss: 0.6078 - val_accuracy: 0.8250
Epoch 5/9
7/7 [==============================] - 0s 43ms/step - loss: 0.3635 - accuracy: 0.8913 - val_loss: 0.5564 - val_accuracy: 0.8300
Epoch 6/9
7/7 [==============================] - 0s 44ms/step - loss: 0.2997 - accuracy: 0.9112 - val_loss: 0.5520 - val_accuracy: 0.8350
Epoch 7/9
7/7 [==============================] - 0s 44ms/step - loss: 0.2486 - accuracy: 0.9212 - val_loss: 0.5429 - val_accuracy: 0.8500
Epoch 8/9
7/7 [===================

7/7 [==============================] - 0s 30ms/step - loss: 1.2742 - accuracy: 0.7912 - val_loss: 0.9969 - val_accuracy: 0.7950
Epoch 3/9
7/7 [==============================] - 0s 28ms/step - loss: 0.7599 - accuracy: 0.8338 - val_loss: 0.6962 - val_accuracy: 0.8100
Epoch 4/9
7/7 [==============================] - 0s 28ms/step - loss: 0.5044 - accuracy: 0.8737 - val_loss: 0.5733 - val_accuracy: 0.8300
Epoch 5/9
7/7 [==============================] - 0s 30ms/step - loss: 0.3879 - accuracy: 0.8925 - val_loss: 0.5349 - val_accuracy: 0.8350
Epoch 6/9
7/7 [==============================] - 0s 28ms/step - loss: 0.3054 - accuracy: 0.9150 - val_loss: 0.5260 - val_accuracy: 0.8500
Epoch 7/9
7/7 [==============================] - 0s 27ms/step - loss: 0.2717 - accuracy: 0.9225 - val_loss: 0.5016 - val_accuracy: 0.8500
Epoch 8/9
7/7 [==============================] - 0s 32ms/step - loss: 0.2316 - accuracy: 0.9475 - val_loss: 0.4990 - val_accuracy: 0.8600
Epoch 9/9
15/15 [===========================

7/7 [==============================] - 0s 29ms/step - loss: 0.9897 - accuracy: 0.8363 - val_loss: 1.0395 - val_accuracy: 0.8300
Epoch 4/9
7/7 [==============================] - 0s 29ms/step - loss: 0.6946 - accuracy: 0.8363 - val_loss: 0.8112 - val_accuracy: 0.8350
Epoch 5/9
7/7 [==============================] - 0s 32ms/step - loss: 0.5274 - accuracy: 0.8612 - val_loss: 0.6890 - val_accuracy: 0.8500
Epoch 6/9
7/7 [==============================] - 0s 35ms/step - loss: 0.4318 - accuracy: 0.8775 - val_loss: 0.6259 - val_accuracy: 0.8500
Epoch 7/9
7/7 [==============================] - 0s 31ms/step - loss: 0.3676 - accuracy: 0.8988 - val_loss: 0.5799 - val_accuracy: 0.8500
Epoch 8/9
7/7 [==============================] - 0s 33ms/step - loss: 0.3254 - accuracy: 0.9100 - val_loss: 0.5577 - val_accuracy: 0.8500
Epoch 9/9
15/15 [==============================] - 0s 3ms/step - loss: 0.5652 - accuracy: 0.8267
Epoch 1/9
7/7 [==============================] - 0s 36ms/step - loss: 2.1653 - accura

Epoch 5/9
7/7 [==============================] - 0s 32ms/step - loss: 0.8682 - accuracy: 0.7675 - val_loss: 0.7626 - val_accuracy: 0.7800
Epoch 6/9
7/7 [==============================] - 0s 35ms/step - loss: 0.6477 - accuracy: 0.7975 - val_loss: 0.6816 - val_accuracy: 0.7650
Epoch 7/9
7/7 [==============================] - 0s 31ms/step - loss: 0.5770 - accuracy: 0.8275 - val_loss: 0.6507 - val_accuracy: 0.8150
Epoch 8/9
7/7 [==============================] - 0s 32ms/step - loss: 0.5037 - accuracy: 0.8450 - val_loss: 0.6243 - val_accuracy: 0.7800
Epoch 9/9
15/15 [==============================] - 0s 3ms/step - loss: 0.5860 - accuracy: 0.8044
Epoch 1/9
7/7 [==============================] - 0s 55ms/step - loss: 2.2353 - accuracy: 0.2400 - val_loss: 2.1436 - val_accuracy: 0.4600
Epoch 2/9
7/7 [==============================] - 0s 35ms/step - loss: 1.9531 - accuracy: 0.5900 - val_loss: 1.8303 - val_accuracy: 0.7000
Epoch 3/9
7/7 [==============================] - 0s 36ms/step - loss: 1.463

7/7 [==============================] - 0s 67ms/step - loss: 0.1813 - accuracy: 0.9475 - val_loss: 0.6055 - val_accuracy: 0.8400
Epoch 7/9
7/7 [==============================] - 0s 67ms/step - loss: 0.1386 - accuracy: 0.9588 - val_loss: 0.5789 - val_accuracy: 0.8400
Epoch 8/9
7/7 [==============================] - 0s 64ms/step - loss: 0.1005 - accuracy: 0.9775 - val_loss: 0.5677 - val_accuracy: 0.8550
Epoch 9/9
15/15 [==============================] - 0s 6ms/step - loss: 0.4692 - accuracy: 0.8733
Epoch 1/9
7/7 [==============================] - 1s 99ms/step - loss: 2.1758 - accuracy: 0.3613 - val_loss: 1.7240 - val_accuracy: 0.6850
Epoch 2/9
7/7 [==============================] - 0s 71ms/step - loss: 1.1767 - accuracy: 0.7525 - val_loss: 0.8403 - val_accuracy: 0.7100
Epoch 3/9
7/7 [==============================] - 1s 74ms/step - loss: 0.5813 - accuracy: 0.8288 - val_loss: 0.7375 - val_accuracy: 0.8200
Epoch 4/9
7/7 [==============================] - 1s 73ms/step - loss: 0.4613 - accura

...

7/7 [==============================] - 2s 235ms/step - loss: 0.1152 - accuracy: 0.9688 - val_loss: 0.5630 - val_accuracy: 0.8700
Epoch 8/9
7/7 [==============================] - 2s 236ms/step - loss: 0.0723 - accuracy: 0.9800 - val_loss: 0.6437 - val_accuracy: 0.8650
Epoch 9/9
15/15 [==============================] - 0s 16ms/step - loss: 0.5770 - accuracy: 0.8667
Epoch 1/9
7/7 [==============================] - 2s 250ms/step - loss: 1.9726 - accuracy: 0.4800 - val_loss: 1.0217 - val_accuracy: 0.7400
Epoch 2/9
7/7 [==============================] - 2s 239ms/step - loss: 0.6740 - accuracy: 0.8075 - val_loss: 0.7245 - val_accuracy: 0.8300
Epoch 3/9
7/7 [==============================] - 2s 237ms/step - loss: 0.5135 - accuracy: 0.8575 - val_loss: 0.5823 - val_accuracy: 0.8400
Epoch 4/9
7/7 [==============================] - 2s 236ms/step - loss: 0.3102 - accuracy: 0.9112 - val_loss: 0.5441 - val_accuracy: 0.8550
Epoch 5/9
7/7 [==============================] - 2s 235ms/step - loss: 0.2241 

7/7 [==============================] - 2s 232ms/step - loss: 0.0798 - accuracy: 0.9800 - val_loss: 0.6442 - val_accuracy: 0.8850
Epoch 9/9
15/15 [==============================] - 0s 16ms/step - loss: 0.6240 - accuracy: 0.8578
Epoch 1/9
7/7 [==============================] - 2s 254ms/step - loss: 1.9851 - accuracy: 0.3600 - val_loss: 0.9684 - val_accuracy: 0.7750
Epoch 2/9
7/7 [==============================] - 2s 236ms/step - loss: 0.6255 - accuracy: 0.8250 - val_loss: 0.9989 - val_accuracy: 0.7600
Epoch 3/9
7/7 [==============================] - 2s 240ms/step - loss: 0.5239 - accuracy: 0.8500 - val_loss: 0.6511 - val_accuracy: 0.8050
Epoch 4/9
7/7 [==============================] - 2s 237ms/step - loss: 0.3171 - accuracy: 0.9100 - val_loss: 0.5388 - val_accuracy: 0.8450
Epoch 5/9
7/7 [==============================] - 2s 235ms/step - loss: 0.2164 - accuracy: 0.9450 - val_loss: 0.5028 - val_accuracy: 0.8700
Epoch 6/9
7/7 [==============================] - 2s 235ms/step - loss: 0.1438 

15/15 [==============================] - 0s 24ms/step - loss: 0.6361 - accuracy: 0.8600
Epoch 1/9
7/7 [==============================] - 2s 259ms/step - loss: 1.9900 - accuracy: 0.4187 - val_loss: 1.0373 - val_accuracy: 0.7500
Epoch 2/9
7/7 [==============================] - 1s 181ms/step - loss: 0.7109 - accuracy: 0.7925 - val_loss: 0.8304 - val_accuracy: 0.8250
Epoch 3/9
7/7 [==============================] - 1s 179ms/step - loss: 0.5187 - accuracy: 0.8550 - val_loss: 0.7345 - val_accuracy: 0.7850
Epoch 4/9
7/7 [==============================] - 1s 179ms/step - loss: 0.3796 - accuracy: 0.8712 - val_loss: 0.5196 - val_accuracy: 0.8600
Epoch 5/9
7/7 [==============================] - 1s 182ms/step - loss: 0.2797 - accuracy: 0.9262 - val_loss: 0.5731 - val_accuracy: 0.8450
Epoch 6/9
7/7 [==============================] - 1s 182ms/step - loss: 0.1892 - accuracy: 0.9450 - val_loss: 0.5864 - val_accuracy: 0.8700
Epoch 7/9
7/7 [==============================] - 1s 179ms/step - loss: 0.1330 

15/15 [==============================] - 0s 12ms/step - loss: 0.5113 - accuracy: 0.8844
Epoch 1/9
7/7 [==============================] - 1s 132ms/step - loss: 2.1574 - accuracy: 0.3663 - val_loss: 1.7482 - val_accuracy: 0.6600
Epoch 2/9
7/7 [==============================] - 1s 117ms/step - loss: 1.2916 - accuracy: 0.7500 - val_loss: 0.8091 - val_accuracy: 0.7250
Epoch 3/9
7/7 [==============================] - 1s 119ms/step - loss: 0.5574 - accuracy: 0.8300 - val_loss: 0.6889 - val_accuracy: 0.8100
Epoch 4/9
7/7 [==============================] - 1s 120ms/step - loss: 0.4765 - accuracy: 0.8737 - val_loss: 0.7793 - val_accuracy: 0.7950
Epoch 5/9
7/7 [==============================] - 1s 122ms/step - loss: 0.3881 - accuracy: 0.8725 - val_loss: 0.8090 - val_accuracy: 0.7650
Epoch 6/9
7/7 [==============================] - 1s 122ms/step - loss: 0.2996 - accuracy: 0.9150 - val_loss: 0.6551 - val_accuracy: 0.8350
Epoch 7/9
7/7 [==============================] - 1s 120ms/step - loss: 0.2779 

7/7 [==============================] - 1s 194ms/step - loss: 1.9754 - accuracy: 0.4275 - val_loss: 1.0186 - val_accuracy: 0.6700
Epoch 2/9
7/7 [==============================] - 1s 195ms/step - loss: 0.6475 - accuracy: 0.8138 - val_loss: 0.7593 - val_accuracy: 0.8100
Epoch 3/9
7/7 [==============================] - 1s 194ms/step - loss: 0.4513 - accuracy: 0.8725 - val_loss: 0.6724 - val_accuracy: 0.8350
Epoch 4/9
7/7 [==============================] - 1s 192ms/step - loss: 0.3279 - accuracy: 0.8963 - val_loss: 0.4969 - val_accuracy: 0.8650
Epoch 5/9
7/7 [==============================] - 1s 193ms/step - loss: 0.2238 - accuracy: 0.9463 - val_loss: 0.5094 - val_accuracy: 0.8800
Epoch 6/9
7/7 [==============================] - 1s 194ms/step - loss: 0.1406 - accuracy: 0.9588 - val_loss: 0.6064 - val_accuracy: 0.8800
Epoch 7/9
7/7 [==============================] - 1s 195ms/step - loss: 0.1036 - accuracy: 0.9613 - val_loss: 0.7435 - val_accuracy: 0.8550
Epoch 8/9
7/7 [======================

In [ ]:
for b in best_models:
    print(b[0].summary(),b[1])